In [1]:
# Input: A zip file with stock information for today downloaded from the NSE web site.
# Output: An excel file with the top movers and most heavily traded stocks of the day.

In [2]:
# Step 1: Download the file.
import urllib.request as urlreq
import os

In [3]:
# The example worked with a hard-coded file name, but I wanted to update this to determine to determine today's date
# programmatically since the host uses a set format for file names.

from datetime import date

year = str(date.today().year)
month = date.today().strftime('%b').upper()
day = str(date.today().day)

baseFileName = 'cm{0}{1}{2}bhav'.format(day, month, year)
baseFileName

'cm17OCT2016bhav'

In [4]:
# Data will be taken from the National Stock Exchange of India, which publishes this data daily.
fileUrl = 'https://www.nseindia.com/content/historical/EQUITIES/{0}/{1}/{2}.csv.zip'.format(year, month, baseFileName)

In [5]:
dataPath = 'stockdata'
if not os.path.isdir(dataPath):
    os.makedirs(dataPath)

In [6]:
zipPath = 'stockdata\\{0}.csv.zip'.format(baseFileName)

In [7]:
# Boilerplate to bypass NSE's script blockers. Mainly sends a user-agent property to mimic a browser.

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
          'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
          'Accept-Charset':'ISO-8859-1;utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding':'none',
          'Accept-Language':'en-US,en;q=0.8',
          'Connection':'keep-alive'}

In [8]:
webRequest = urlreq.Request(fileUrl,headers=header)

In [9]:
try:
    # The page opens as a file, so we should receive a zip from this.
    # Any possible HTTPError is handled in the except block below.
    page = urlreq.urlopen(webRequest)
    content = page.read()
    output = open(zipPath,'wb')
    output.write(bytearray(content))
    output.close()
except urlreq.HTTPError as e:
    print(e.fp.read())
    print("Looks like the download did not go through for url ",fileUrl)

In [10]:
# Step 2: Extract the file.
import zipfile

extractPath = dataPath

In [11]:
if os.path.exists(zipPath):
    print("Found zip file, proceeding...")
    extractedFiles = [] # May find multiple files.
    fh = open(zipPath,'rb')
    zipHandler = zipfile.ZipFile(fh)
    for filename in zipHandler.namelist():
        zipHandler.extract(filename,extractPath)
        extractedFiles.append(extractPath + '\\' + filename)
        print("Extracted " + filename + " from zipfile to " + (extractPath + '\\' + filename))
    print("Extracted " + str(len(extractedFiles)) + " file(s) from zip archive.")

Found zip file, proceeding...
Extracted cm17OCT2016bhav.csv from zipfile to stockdata\cm17OCT2016bhav.csv
Extracted 1 file(s) from zip archive.


In [12]:
# Step 3: Parse the file for info.
import csv

In [13]:
# We know it only has one file, so extract the first from the list.
fileName = extractedFiles[0]
lineNum = 0

#Each stock has data on how much itmoved, and how heavily it was traded. That will be tracked here.
stockData = []
with open(fileName,'r') as csvfile:
    #quotechar specified in case of internal lists, which will be enclosed in double quotes.
    lineReader = csv.reader(csvfile,delimiter=',',quotechar='"')
    for row in lineReader:
        lineNum = lineNum + 1
        # Check for header row, and skip if that's the case. (We don't need the column names).
        if lineNum == 1:
            print("File opened. Parsing lines...")
            continue
        # We're looking for column 0 (stock ticker), column 5 (last closing price), column 7 (yesterday's close), column 9 (traded value in rupees).
        symbol = row[0]
        close = row[5]
        prevClose = row[7]
        tradedQty = row[9]
        pctChange = float(close)/float(prevClose) - 1
        stockInfo = [symbol, pctChange, float(tradedQty)]
        stockData.append(stockInfo)
        print(symbol, "{:,.1f}".format(float(tradedQty)/1e6) + "M INR", "{:,.1f}".format(pctChange*100) + "%")
    print("Done parsing contents - closing file...")
print("Processed info for " + str(lineNum) + " stock(s).")

File opened. Parsing lines...
20MICRONS 6.9M INR 0.7%
3IINFOTECH 14.0M INR 3.4%
3MINDIA 8.0M INR -0.1%
8KMILES 31.2M INR -2.9%
A2ZINFRA 10.6M INR -2.1%
AARTIDRUGS 14.3M INR -2.1%
AARTIIND 23.5M INR -2.7%
AARVEEDEN 1.7M INR 1.3%
ABAN 2,818.2M INR 6.0%
ABB 42.7M INR -0.1%
ABBOTINDIA 7.7M INR 0.1%
ABFRL 65.5M INR -0.0%
ABGSHIP 17.6M INR 1.4%
ABIRLANUVO 451.4M INR -2.4%
ACC 527.6M INR -2.2%
ACCELYA 2.8M INR 0.3%
ACE 2.7M INR -1.8%
ACROPETAL 0.0M INR 3.3%
ADANIENT 207.2M INR -0.3%
ADANIPORTS 417.4M INR -1.4%
ADANIPOWER 350.9M INR 1.7%
ADANITRANS 117.0M INR 2.1%
ADFFOODS 7.6M INR 0.5%
ADHUNIK 0.4M INR 0.0%
ADHUNIKIND 3.0M INR -0.7%
ADI 4.7M INR 0.2%
ADLABS 86.3M INR 1.1%
ADORWELD 0.8M INR -0.8%
ADSL 31.3M INR 7.3%
ADVANIHOTR 0.1M INR -0.6%
ADVENZYMES 629.3M INR 1.8%
AEGISCHEM 26.2M INR -1.8%
AFL 1.5M INR -0.9%
AGARIND 18.2M INR 5.4%
AGCNET 2.0M INR 1.3%
AGRITECH 0.2M INR 4.9%
AGRODUTCH 0.0M INR -4.8%
AHLEAST 0.3M INR 0.5%
AHLUCONT 3.4M INR -2.1%
AHLWEST 0.0M INR 0.0%
AIAENG 10.4M INR 0.2%
AI

In [14]:
# Step 4: Sort the data.

# The 'sorted' keyword can organize any iterable by a given function, specified by 'key'.
stockDataSortedByQty = sorted(stockData, key=lambda x:x[2], reverse=True)
stockDataSortedByPct = sorted(stockData, key=lambda x:x[1], reverse=True)

In [15]:
stockDataSortedByQty

[['ICICIBANK', 0.0703205791106516, 16081519358.05],
 ['SBIN', 0.004363347877826218, 4284785156.7],
 ['AXISBANK', -0.0031691155286660555, 4165807398.7],
 ['INFY', -0.004624446283405637, 4121892312.8],
 ['IOC', -0.0010886469673406785, 3497500006.1],
 ['TCS', -0.0015848195418814992, 3221632238.65],
 ['ZEEL', -0.0489120151371808, 3176695553.95],
 ['DHFL', 0.02075722351378273, 2979725330.8],
 ['ABAN', 0.060478468899521465, 2818199180.0],
 ['HDFC', -0.010713878652026776, 2785928353.65],
 ['NILKAMAL', 0.0880428885981428, 2766209259.15],
 ['TATASTEEL', -0.0036381275770069887, 2765076999.05],
 ['ULTRACEMCO', -0.005859036293818898, 2598796965.25],
 ['RELIANCE', -0.015775066116085923, 2400217761.5],
 ['MARUTI', -0.01605321196038967, 2254290683.6],
 ['LT', -0.018239142963691113, 2080302523.55],
 ['PETRONET', 0.003993510545363899, 2065742968.75],
 ['CEATLTD', -0.06445866356431318, 2050177790.85],
 ['TATAMOTORS', -0.015374932566085353, 1907600422.6],
 ['VEDL', -0.0137966274910577, 1698481506.4],
 ['

In [16]:
stockDataSortedByPct

[['AURDIS', 0.19999999999999996, 1512000.0],
 ['PROZONINTU', 0.19999999999999996, 37748320.6],
 ['SRGINFOTEC', 0.19999999999999996, 3876.25],
 ['TIL', 0.19982018431108117, 65563209.1],
 ['DIAPOWER', 0.19974715549936795, 27531401.8],
 ['DCM', 0.19959162838182753, 134491464.25],
 ['MIRCELECTR', 0.19787985865724367, 49318041.65],
 ['PARABDRUGS', 0.19718309859154926, 3358313.7],
 ['ESSARSHPNG', 0.1968911917098446, 217394517.4],
 ['COMPUSOFT', 0.1966527196652721, 16954518.05],
 ['LML', 0.196078431372549, 130303571.3],
 ['MUKANDLTD', 0.19444444444444442, 335.4],
 ['DIGJAMLTD', 0.188405797101449, 57694579.45],
 ['MAGNUM', 0.18749999999999978, 125938.45],
 ['FIEMIND', 0.17285772857728565, 248339155.9],
 ['VARDMNPOLY', 0.1594202898550725, 66252833.9],
 ['GUFICBIO', 0.14565826330532206, 20153748.35],
 ['LPDC', 0.13698630136986312, 295030.65],
 ['RECLTD', 0.13492063492063489, 7150000.0],
 ['PROVOGE', 0.12790697674418605, 7188828.6],
 ['NICCO', 0.125, 4691.5],
 ['RAJOIL', 0.125, 105731.45],
 ['HIN

In [17]:
# Step 5: Export to Excel file.

import xlsxwriter

In [18]:
excelFilePath = 'stockdata\\{0}.xlsx'.format(baseFileName)

In [19]:
workbook = xlsxwriter.Workbook(excelFilePath)
worksheet = workbook.add_worksheet("Summary")

worksheet.write_row("A1",["Top Traded stocks"])
worksheet.write_row("A2",["Stock","% Change","Value Traded (INR)"])

0

In [20]:
# To populate, we'll pikcup the top 5 stocks in the category.
for lineNum in range(5):
    lineToWrite = stockDataSortedByQty[lineNum]
    worksheet.write_row("A" + str(lineNum + 3), lineToWrite)
    
workbook.close()